# data.world
## 2010-2015 NYC Crime Data

> #### Setup  

> Before running data.world notebooks for the first time, you'll need to:  
1. Install data.world's Python package, including optional `pandas` dependencies: 
```shell
pip install git+git://github.com/datadotworld/data.world-py.git#egg=project[pandas]
```
1. Obtain an API access token at https://data.world/settings/advanced
1. Store API access token using the `dw` command-line tool: 
```shell
dw configure
```

> Once your environment is set up, these steps do not need to be repeated for other data.world notebooks.

In [ ]:
%matplotlib inline
import warnings; warnings.simplefilter('ignore')

import datadotworld as dw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
from datetime import timedelta, date
import datetime

In [ ]:
# Datasets are referenced by their path
dataset_key = 'carolinadata/2010-2015-nyc-crime-data'

# Or simply by their URL
dataset_key = 'https://data.world/carolinadata/2010-2015-nyc-crime-data'

In [ ]:
# Load dataset (onto the local file system)
dataset_local = dw.load_dataset(dataset_key)  # cached under ~/.dw/cache

In [ ]:
# See what is in it
dataset_local.describe('nypd_complaint_data_historic_short')

In [ ]:
#Using pandas dataframes
nyc = dataset_local.dataframes['nypd_complaint_data_historic_short']

#Ensure dates are actually dates
nyc['cmplnt_fr_dt'] = pd.to_datetime(nyc['cmplnt_fr_dt'], errors = 'coerce')
nyc[['latitude', 'longitude']]

moon = pd.read_csv('moon.csv')
moon['date'] = pd.to_datetime(moon['date'], errors = 'coerce')

#Max value of hexbin bin over entire data, so bins are relative across plots
maxVal = 0

In [ ]:
nyc.head()

In [ ]:
moon.head()

In [ ]:
#The map
bm = Basemap(projection='mill',llcrnrlat=40.47,urcrnrlat=40.92,\
        llcrnrlon=-74.26, urcrnrlon=-73.7, lat_ts=40.5,resolution='f')

In [ ]:
def createNYCMap(x,y,t=None, m = bm):
    global maxVal
    
    gs = gridspec.GridSpec(4,2,
                       width_ratios=[5,1])
    
    plt.subplot(gs[:, 0])
    #m = Basemap(projection='mill',llcrnrlat=40.47,urcrnrlat=40.92,\
    #    llcrnrlon=-74.26, urcrnrlon=-73.7, lat_ts=40.5,resolution=res)
    
    mx,my = m(y,x)

    hb = m.hexbin(mx,my, gridsize=50, 
                  C=np.ones_like(y, dtype=np.float) / maxVal,
                  reduce_C_function=np.sum,
                  mincnt = 1, cmap=plt.cm.jet, bins='log')

    maxVal = max(hb.get_array().max(), maxVal)
    m.drawcoastlines()
    m.fillcontinents(zorder = 0)
    m.drawcountries(linewidth=.2)
    if t is not None:
        plt.title(t['phase'] + ' of ' + t['date'].strftime("%B %d, %Y"))
    
    for i in range(0,4):
        plt.subplot(gs[i, 1])
        img = mpimg.imread(str(i+1) + '.png')
        if i+1 == t['phaseid']:
            fig = plt.imshow(img)
        else:
            fig = plt.imshow(img, alpha = .4)   
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

    fig1 = plt.gcf()
    plt.show()
    fig1.savefig('figis/' + str(t['date']) + '.png')
    return plt

In [ ]:
#Util functions, seem pretty self-explanatory
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
def location_of_complaints_on_date(d):
    return nyc[nyc['cmplnt_fr_dt'] == d][['latitude', 'longitude']].dropna(subset=['latitude', 'longitude'])

def location_of_complaints_around_moons(dates):
    date_list = []
    for d in dates:
        date_list += [d - datetime.timedelta(days=x) for x in range(0, 4)] + [d + datetime.timedelta(days=x) for x in range(1, 4)]
    return nyc[nyc['cmplnt_fr_dt'].isin(date_list)][['latitude', 'longitude']].dropna(subset=['latitude', 'longitude'])

def location_of_complaints_around_moon(d):
    date_list = [d - datetime.timedelta(days=x) for x in range(0, 4)] + [d + datetime.timedelta(days=x) for x in range(1, 4)]
    return nyc[nyc['cmplnt_fr_dt'].isin(date_list)][['latitude', 'longitude']].dropna(subset=['latitude', 'longitude'])

def to_lists(df):
    return np.array(df['latitude']), np.array(df['longitude'])

def quick_plot(row):
    d = row['date']
    x,y = to_lists(location_of_complaints_around_moon(d))
    return createNYCMap(x,y,t=row)

start_date = date(2010, 1, 1)
end_date = date(2015, 12, 31)

In [ ]:
#must run this one first to correctly set maxVal
allcomplaints = moon['date'].tolist()
x,y = to_lists(location_of_complaints_around_moons(allcomplaints))
createNYCMap(x,y)

In [ ]:
#Generate each plot
for _, row in moon.iterrows():
    quick_plot(row)

In [ ]:
#Below require a little change to map function because I
#broke it right at the end of the challenge and won't be fixing it.

In [ ]:
#Summation of all full moons
fulls = moon[moon['phase']=='Full Moon']['date'].tolist()
x,y = to_lists(location_of_complaints_around_moons(fulls))
createNYCMap(x,y)

In [ ]:
fulls = moon[moon['phase']=='Full Moon']['date'].tolist()
x,y = to_lists(location_of_complaints_around_moons(fulls))
createNYCMap(x, y, res = 'l')

In [ ]:
news = moon[moon['phase']=='New Moon']['date'].tolist()
x,y = to_lists(location_of_complaints_around_moons(news))
createNYCMap(x, y, res = 'l')

In [ ]:
fq = moon[moon['phase']=='First Quarter']['date'].tolist()
x,y = to_lists(location_of_complaints_around_moons(fq))
createNYCMap(x, y, res = 'l')

In [ ]:
lq = moon[moon['phase']=='Last Quarter']['date'].tolist()
x,y = to_lists(location_of_complaints_around_moons(lq))
createNYCMap(x, y, res = 'l')